In [30]:
import pandas as pd
import os

# Encontra diretorio atual
atual_dir = os.getcwd()

# Acessa arquivo pkl das noticias
parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])

parent_dir = os.path.split(parent_dir[0])

caminho_pkl = os.path.join(parent_dir[0], "Pre-processamento\\noticias_pre_processadas_df.pkl")

# Carregar dataframe salvo em formato pickle
df = pd.read_pickle(caminho_pkl)

# Total de classes
classes = df['label'].nunique()


In [31]:
import string
import spacy
import unicodedata
from tqdm import tqdm
import numpy as np

# Habilita suporte do tqdm para os métodos de progressão do pandas (como progress_aplly)
tqdm.pandas()

# Carrega modelo de linguagem 'pt_core_news_lg' do spacy para processamento de texto em português
# Desabilita os componentes 'parser' e 'ner', já que não são necessários para a lematização
modelo_spacy_nlp = spacy.load("pt_core_news_lg", disable=["parser", "ner"])

def preprocess_data(df, coluna_texto):
    """
    Realiza o pré-processamento dos dados de um texto em um Dataframe do Pandas.
    Remove pontuação, números e palavras comuns (stop words), converte para minúsculas, remove 
    acentos e símbolos diversos, e aplica lematização.
    """

    # Remove pontuação
    print("Removendo pontuação...")
    traducao = str.maketrans('', '', string.punctuation)
    df[coluna_texto] = df[coluna_texto].progress_apply(lambda x: x.translate(traducao))

    # Remove números
    print("Removendo números...")
    traducao = str.maketrans('', '', string.digits)
    df[coluna_texto] = df[coluna_texto].progress_apply(lambda x: x.translate(traducao))

    # Remove acentos e símbolos diversos
    print("Removendo acentos e símbolos diversos...")
    def remove_acentos_e_simbolos(text):
        try:
            # Normaliza a string para a forma NFKD e mantém apenas caracteres que não são diacríticos
            # nem combinam caracteres com diacríticos
            return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c) and unicodedata.category(c) != 'Mn')
        except TypeError:
            # Se não for possível normalizar um caractere, retorna uma string vazia
            return ''
    df[coluna_texto] = df[coluna_texto].progress_apply(lambda x: remove_acentos_e_simbolos(x))
    
    # Converte para minúsculas
    print("Convertendo para minúsculas...")
    df[coluna_texto] = df[coluna_texto].progress_apply(lambda x: x.lower())

    # Lematização
    print("Computando Lematização...")
    def lematizar_texto(doc):
        return [token.lemma_ for token in doc if not token.is_stop]

    n_chunks = 10  # Ajuste esse valor de acordo com o tamanho da base de dados e a memória disponível no sistema
    chunks = np.array_split(df, n_chunks) # Divide o dataframe em várias partes

    chunks_processados = []
    for i, chunk in enumerate(chunks):
        print(f"Processando segmento {i + 1} de {n_chunks}")
        chunk_processado = chunk.copy() # Cria uma cópia para realizar o processamento
        
        # Aplica a função 'lematizar_texto' a cada documento processado pelo spaCy (usando 'spacy_nlp_model.pipe') e
        # atribui os resultados (uma lista de palavras lematizadas) à coluna 'coluna_texto' do DataFrame 'chunks_processados'.
        # O tqdm é utilizado para exibir uma barra de progresso durante o processamento dos documentos.
        chunk_processado[coluna_texto] = [lematizar_texto(doc) for doc in tqdm(modelo_spacy_nlp.pipe(chunk[coluna_texto].astype(str), batch_size=100, disable=['parser', 'ner']), total=len(chunk[coluna_texto]))]

        # Junta as partes em uma lista, para formar o dataframe final
        chunks_processados.append(chunk_processado)

    concatenated_df = pd.concat(chunks_processados) # Concatenar os DataFrames processados

    df[coluna_texto] = concatenated_df[coluna_texto] # Atribuir a coluna 'texto' processada de volta ao dataframe original
    
    # Remover tokens com espaços vazios
    print("Remover tokens com espaços vazios...")
    df['Texto'] = df['Texto'].progress_apply(lambda x: [token for token in x if token.strip()])

In [32]:
import pickle

import os

# Encontra diretorio atual
atual_dir = os.getcwd()

# Acessa arquivo pkl das noticias
parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])

parent_dir = os.path.split(parent_dir[0])

caminho = os.path.join(parent_dir[0], "Modelos\\GLoVE\\Treinamento\\tokenizer.pkl")

# Carrega o tokenizador de um arquivo
with open(caminho, 'rb') as file:
    tokenizer = pickle.load(file)


In [33]:
import pickle
import os

# Carregar a lista 'sequences' de um arquivo usando pickle
# Encontra diretorio atual
atual_dir = os.getcwd()

# Acessa arquivo pkl das noticias
parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])


caminho_pkl = os.path.join(parent_dir[0], "Word2Vec\\sequences.pickle")

with open(caminho_pkl, 'rb') as handle:
    sequences = pickle.load(handle)

In [34]:
import numpy as np

# Calcula o comprimento de cada sequência em 'sequences' e armazena os resultados na lista 'sequence_lengths'
sequence_lengths = [len(seq) for seq in sequences]

# Calcula a média dos comprimentos das sequências usando a função 'mean' do numpy
mean_length = np.mean(sequence_lengths)

# Obtém o comprimento máximo das sequências usando a função 'max' do Python
max_length = max(sequence_lengths)

# Imprime a média e o comprimento máximo das sequências
print("Mean sequence length: ", mean_length)
print("Maximum sequence length: ", max_length)

Mean sequence length:  173.9763956201759
Maximum sequence length:  3835


In [35]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Calcula uma porcentagem de 95% dos comprimentos das sequências (preserva 95% das sequências)
max_length = int(np.percentile(sequence_lengths, 95))

# Realiza o preenchimento das sequências com base no novo 'max_length'
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [36]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import os

# Carregar a lista 'sequences' de um arquivo usando pickle
# Encontra diretorio atual
atual_dir = os.getcwd()

# Acessa arquivo pkl das noticias
parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])

caminho_modelo = os.path.join(parent_dir[0], "GLoVE\\Treinamento")

# Carrega arquivos pertinentes ao treinamento da rede neural e X_test e y_test (usados na avaliação de resultados)
model = load_model(caminho_modelo + '\\modelo_BiLSTM_glove6b100d.h5')
X_test = np.load(caminho_modelo + "\\X_test_BiLSTM_glove6b100d.npy")
y_test = np.load(caminho_modelo + "\\y_test_BiLSTM_glove6b100d.npy")

In [37]:
# Dataframe simples com uma notícia só para aplicar a função preprocess_data

import pandas as pd
from nltk.tokenize import word_tokenize

value = """Conselho de Segurança da ONU aprova resolução de cessar-fogo imediato em Gaza
Texto aprovado estipula que a pausa aconteça durante o Ramadã, que começou em 10 de março e vai até 9 de abril. Documento é de cumprimento obrigatório, mas ONU não tem poder coercitivo. Secretário-geral da ONU, Antonio Guterres, pediu que o governo israelense acatasse a decisão do conselho.
O Conselho de Segurança da Organização das Nações Unidas (ONU) aprovou nesta segunda-feira (25) uma resolução de cessar-fogo imediato na Faixa de Gaza.
A resolução, feita por um grupo de dez países rotativos liderados por Moçambique, é a primeira que o conselho conseguiu aprovar sobre um cessar-fogo no território palestino.

Israel promove uma ofensiva militar no território palestino desde que terroristas do Hamas invadiram o território israelense e mataram centenas de pessoas, em outubro de 2023.

A aprovação, no entanto, não uma solução para a guerra. O desafio agora é garantir que os atores envolvidos nela - o governo de Israel e o grupo terrorista - cumpram as determinações exigidas no texto da ONU.

Isso porque, embora as resoluções do Conselho de Segurança sejam juridicamente vinculativas, na prática acabam ignoradas por muitos países.

O secretário-geral da ONU, Antonio Guterres, pediu que o governo israelense acatasse a decisão do conselho.

Do que se trata
O texto determina um cessar-fogo durante o mês do Ramadã, o período sagrado para os muçulmanos — que começou dia 10 e termina em 9 de abril—, mas pede que a trégua aumente até virar permanente.

A resolução também pede a "libertação imediata e incondicional de reféns" e “a necessidade urgente de expandir o fluxo” de ajuda humanitária para Gaza.
O Conselho de Segurança é formado por 15 países: cinco com assento permanente (China, França, Rússia, Reino Unido e Estados Unidos) e dez rotativos (Argélia, Equador, Guiana, Japão, Malta, Moçambique, Coréia do Sul, Serra Leoa, Eslovênia e Suíça).

Na semana passada, uma resolução dos EUA pedindo a pausa nos bombardeios foi vetada pela China e pela Rússia, que estão entre os cinco membros permanentes do Conselho de Segurança e, por isso, têm poder de veto.

Proposta dos EUA vetada
A proposta dos EUA, uma mudança de posição de Washington na guerra entre Israel e Hamas, previa o cessar-fogo imediato na guerra entre Israel e o grupo terrorista Hamas, além da libertação de reféns.

Ao justificar seu voto, o embaixador da Rússia na ONU, Vassily Nebenzia, acusou os EUA de falsas promessas e de só reconhecer a necessidade de um cessar-fogo "quando mais de 30 mil habitantes de Gaza já morreram".
"""

# Crie um DataFrame com uma linha e a coluna 'data'
df_predict = pd.DataFrame(data={'Texto': [value]})

# Faz pré-processamento
preprocess_data(df_predict, 'Texto')

# Conversão dos dados para serem usados no modelo (rede neural)
preprocessed_articles = df_predict['Texto'].tolist()

sequences_test = []
for text in tqdm(preprocessed_articles):
    # Converte o texto em sequências de tokens usando o tokenizer
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequences_test.append(sequence)
    
padded_example = pad_sequences(sequences_test, maxlen=max_length, padding='post')

# Fazer a previsão usando o modelo
predictions = model.predict(padded_example)

Removendo pontuação...


100%|██████████| 1/1 [00:00<00:00, 988.06it/s]


Removendo números...


100%|██████████| 1/1 [00:00<?, ?it/s]


Removendo acentos e símbolos diversos...


...

Processando segmento 10 de 10


0it [00:00, ?it/s]


Remover tokens com espaços vazios...


100%|██████████| 1/1 [00:00<?, ?it/s]


1/1 [==============================] - 2s 2s/step


In [38]:
predictions

array([[0.14617221, 0.8538278 ]], dtype=float32)

In [39]:
# Identificar a classe com a maior probabilidade
predicted_class = np.argmax(predictions)

# Exibir a classe prevista
print(f"Classe prevista: {predicted_class}")

Classe prevista: 1


In [40]:
import pandas as pd
import os

# Encontra diretorio atual
atual_dir = os.getcwd()

# Acessa arquivo csv das noticias
parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])

parent_dir = os.path.split(parent_dir[0])

caminho_csv = os.path.join(parent_dir[0], "Pre-processamento\\noticias_dados_limpos.csv")

# Carregar dataframe salvo em formato csv
df = pd.read_csv(caminho_csv)

from sklearn.preprocessing import LabelEncoder

# Cria um objeto LabelEncoder
le = LabelEncoder()

# Transforma os labels para variáveis categóricas
df['label'] = le.fit_transform(df['Categoria'])

original_class = le.inverse_transform([predicted_class]) 
print(f"Classe original: {original_class[0]}")

Classe original: Real


In [41]:
# Dataframe simples com uma notícia só para aplicar a função preprocess_data

import pandas as pd
from nltk.tokenize import word_tokenize

value = """
"Estudo Inovador Afirma que Ficar de Pijama o Dia Todo Aumenta a Produtividade"

Uma pesquisa recente realizada por uma equipe de psicólogos da Universidade de Bem-Estar Mental revelou uma descoberta surpreendente: ficar de pijama durante o dia pode aumentar significativamente a produtividade.

Segundo os pesquisadores, vestir pijamas cria uma sensação de conforto e relaxamento, o que reduz os níveis de estresse e ansiedade. Isso, por sua vez, permite que as pessoas se concentrem mais em suas tarefas e sejam mais eficientes em seu trabalho.

O estudo, que acompanhou mais de mil participantes ao longo de seis meses, descobriu que aqueles que adotaram a prática de trabalhar de pijama relataram uma melhoria notável em sua capacidade de concentração e tomada de decisões. Além disso, muitos participantes relataram sentir-se mais motivados e criativos em suas atividades diárias.

Essa descoberta desafia a noção convencional de que vestir roupas formais é essencial para o sucesso profissional. Os pesquisadores agora estão explorando maneiras de integrar essa prática inovadora em ambientes de trabalho tradicionais, potencialmente revolucionando a cultura corporativa.

Portanto, se você está procurando aumentar sua produtividade, talvez seja hora de abandonar o terno e adotar o conforto do pijama durante o expediente!
"""

# Crie um DataFrame com uma linha e a coluna 'data'
df_predict = pd.DataFrame(data={'Texto': [value]})

# Faz pré-processamento
preprocess_data(df_predict, 'Texto')

# Conversão dos dados para serem usados no modelo (rede neural)
preprocessed_articles = df_predict['Texto'].tolist()

sequences_test = []
for text in tqdm(preprocessed_articles):
    # Converte o texto em sequências de tokens usando o tokenizer
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequences_test.append(sequence)
    
padded_example = pad_sequences(sequences_test, maxlen=max_length, padding='post')

# Fazer a previsão usando o modelo
predictions = model.predict(padded_example)

Removendo pontuação...


100%|██████████| 1/1 [00:00<00:00, 998.41it/s]


Removendo números...


100%|██████████| 1/1 [00:00<00:00, 999.60it/s]


Removendo acentos e símbolos diversos...


...

Processando segmento 10 de 10


0it [00:00, ?it/s]


Remover tokens com espaços vazios...


100%|██████████| 1/1 [00:00<?, ?it/s]


1/1 [==============================] - 0s 137ms/step


In [42]:
predictions

array([[0.9948832 , 0.00511682]], dtype=float32)

In [43]:
# Identificar a classe com a maior probabilidade
predicted_class = np.argmax(predictions)

# Exibir a classe prevista
print(f"Classe prevista: {predicted_class}")

Classe prevista: 0


In [44]:
import pandas as pd
import os

# Encontra diretorio atual
atual_dir = os.getcwd()

# Acessa arquivo csv das noticias
parent_dir = os.path.split(atual_dir)

parent_dir = os.path.split(parent_dir[0])

parent_dir = os.path.split(parent_dir[0])

caminho_csv = os.path.join(parent_dir[0], "Pre-processamento\\noticias_dados_limpos.csv")

# Carregar dataframe salvo em formato csv
df = pd.read_csv(caminho_csv)

from sklearn.preprocessing import LabelEncoder

# Cria um objeto LabelEncoder
le = LabelEncoder()

# Transforma os labels para variáveis categóricas
df['label'] = le.fit_transform(df['Categoria'])

original_class = le.inverse_transform([predicted_class]) 
print(f"Classe original: {original_class[0]}")

Classe original: Falso
